## Imports

In [1]:
import os

from pandas import read_csv

from sklearn.model_selection import train_test_split

from autokeras import StructuredDataClassifier

## Main

In [2]:
# Load dataset
data_path = '../../Data/wine_data.csv'
df = read_csv(data_path)
print(df.shape)

(178, 14)


In [3]:
df.head(10)

,inputs Alcohol,MalicAcid,Ash,AlcalinityOfAsh,Magnesium,TotalPhenols,flavanoids,NonflavanoidsPhenols,Proanthocyanins,ColorIntensity,Hue,OD280/OD315,Proline,Cultivar
0,12.08,1.33,2.30,23.6,70,2.20,1.59,0.42,1.38,1.74,1.07,3.21,625,1
1,12.08,1.13,2.51,24.0,78,2.00,1.58,0.40,1.40,2.20,1.31,2.72,630,1
2,12.37,1.17,1.92,19.6,78,2.11,2.00,0.27,1.04,4.68,1.12,3.48,510,1
3,13.11,1.01,1.70,15.0,78,2.98,3.18,0.26,2.28,5.30,1.12,3.18,502,1
4,12.04,4.30,2.38,22.0,80,2.10,1.75,0.42,1.35,2.60,0.79,2.57,580,1
5,12.25,1.73,2.12,19.0,80,1.65,2.03,0.37,1.63,3.40,1.00,3.17,510,1
6,12.69,1.53,2.26,20.7,80,1.38,1.46,0.58,1.62,3.05,0.96,2.06,495,1
7,12.77,3.43,1.98,16.0,80,1.63,1.25,0.43,0.83,3.40,0.70,2.12,372,1
8,13.88,5.04,2.23,20.0,80,0.98,0.34,0.40,0.68,4.90,0.58,1.33,415,2
9,12.08,1.83,2.32,18.5,81,1.60,1.50,0.52,1.64,2.40,1.08,2.27,480,1


In [4]:
# Split into input and output elements
X = df.drop('Cultivar', axis=1).values
y = df.Cultivar.values

print(X.shape, y.shape)

(178, 13) (178,)


In [5]:
# Preprocessing

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=70, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(108, 13) (70, 13) (108,) (70,)


In [6]:
# Searching

search = StructuredDataClassifier(max_trials=30)

search.fit(x=X_train, y=y_train, verbose=1)

Trial 30 Complete [00h 01m 07s]
val_accuracy: 1.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 29m 59s
INFO:tensorflow:Oracle triggered exit
Epoch 1/19
4/4 [==============================] - 1s 2ms/step - loss: 0.9780 - accuracy: 0.5293
Epoch 2/19
4/4 [==============================] - 0s 2ms/step - loss: 0.9079 - accuracy: 0.6405
Epoch 3/19
4/4 [==============================] - 0s 2ms/step - loss: 0.8452 - accuracy: 0.7354
Epoch 4/19
4/4 [==============================] - 0s 2ms/step - loss: 0.7857 - accuracy: 0.8198
Epoch 5/19
4/4 [==============================] - 0s 2ms/step - loss: 0.7290 - accuracy: 0.8623
Epoch 6/19
4/4 [==============================] - 0s 3ms/step - loss: 0.6741 - accuracy: 0.9131
Epoch 7/19
4/4 [==============================] - 0s 3ms/step - loss: 0.6211 - accuracy: 0.9189
Epoch 8/19
4/4 [==============================] - 0s 3ms/step - loss: 0.5697 - accuracy: 0.9278
Epoch 9/19
4/4 [==============================] - 0s 2ms/step - loss: 0.5202 - ac

In [7]:
# Evaluation
loss, acc = search.evaluate(X_test, y_test, verbose=1)
print(f'Accuracy: {acc}')

3/3 [==============================] - 0s 2ms/step - loss: 0.1607 - accuracy: 0.9857
Accuracy: 0.9857142567634583


In [8]:
# Get the best performing model
model = search.export_model()

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 13)]              0         
_________________________________________________________________
multi_category_encoding (Mul (None, 13)                0         
_________________________________________________________________
normalization (Normalization (None, 13)                27        
_________________________________________________________________
dense (Dense)                (None, 32)                448       
_________________________________________________________________
re_lu (ReLU)                 (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32)                0     

In [9]:
# Saving the model

if not os.path.isdir('model'):
    os.mkdir('model')

model.save('model/wine.h5')